# Two-particle self consistency (TPSC)

$$ \chi_{sp}(i\omega_n, k) \equiv \frac{\chi_0(i\omega_n, k)}{1 - \frac{U_{sp}}{2} \chi_0(i\omega_n, k)} $$

$$ \chi_{ch}(i\omega_n, k) \equiv \frac{\chi_0(i\omega_n, k)}{1 + \frac{U_{ch}}{2} \chi_0(i\omega_n, k)} $$

$$ \mathrm{Tr} [ \chi ] \equiv \frac{1}{\beta N_k} \sum_{n, k} \chi(i\omega_n, k) $$

In [ ]:
# Reload beta, chi  ....


In [ ]:
def chi_wk_trace(chi_wk):
    wmesh, kmesh = chi_wk.mesh.components
    tr_chi_wk = chi_wk.data.sum() / len(kmesh) / wmesh.beta # tail correction FIXME!!!
    assert(np.abs(tr_chi_wk.imag) < 1e-10)
    return tr_chi_wk.real

def Usp_root_problem(Usp, chi0, n, U):

    tr_chi_sp = chi_wk_trace(chi_wk_from_U_and_chi0_wk(chi0, U=0.5*Usp))
    diff = tr_chi_sp + 0.5 * Usp/U * n**2 - n

    return diff

def Uch_root_problem(Uch, chi0, n, U, docc):

    tr_chi_ch = chi_wk_trace(chi_wk_from_U_and_chi0_wk(chi0, U=-0.5*Uch))
    diff = tr_chi_ch - 2 * docc - n + n**2

    return diff

def solve_Usp_and_Uch(chi0, U, n, Usp0=0.1, Uch0=0.1):

    from scipy.optimize import fsolve
    
    Usp = fsolve(Usp_root_problem, Usp0, args=(chi0, n, U), xtol=1e-2)[0]
    docc = 0.25 * Usp / U * n**2        
    Uch = fsolve(Uch_root_problem, Uch0, args=(chi0, n, U, docc), xtol=1e-2)[0]

    return Usp, Uch, docc

In [ ]:
n = 1.0
Usp, Uch = 0.47, 0.37 # Initial guess

U_vec = np.concatenate((np.arange(0.3, 1., 0.2), np.arange(1., 6., 1.)))    
Usp_vec, Uch_vec, docc_vec = [np.zeros_like(U_vec) for x in xrange(3)]

print ''.join('| %-11s' % s for s in ['n', 'U', 'Usp', 'Uch', 'docc']), '|'
print '-'*67

for idx, U in enumerate(U_vec):
    Usp, Uch, docc = solve_Usp_and_Uch(chi0_wk, U, n, Usp0=Usp, Uch0=Uch)
    Usp_vec[idx], Uch_vec[idx], docc_vec[idx] = Usp, Uch, docc
    print ''.join('| %4.4E ' % x for x in [n, U, Usp, Uch, docc]), '|'

In [ ]:
plt.figure(figsize=(3.25*2, 5))

plt.title(r'$\beta = %2.2f$' % beta)
plt.plot(U_vec, Usp_vec, 'o-', label=r'$U_{sp}$', alpha=0.5)
plt.plot(U_vec, Uch_vec, 'o-', label=r'$U_{ch}$', alpha=0.5)

plt.ylim([0, 20]); plt.xlim([0, 5])
plt.legend(loc='best'); plt.xlabel(r'$U$')
plt.tight_layout()
plt.savefig('figure_Usp_and_Uch_vs_U.pdf')

# TPSC and the Mermin-Wagner theorem

Temperature sweep for $U=4$

Spin structure factor 

$$S(k) \equiv \sum_n \chi_{sp}(i\omega_n, k)$$

In [ ]:
U = 4.

T_rpa_vec = np.concatenate((np.arange(10., 3., -1.), np.arange(3., 0.75, -0.2)))
S_rpa_vec = np.zeros_like(T_rpa_vec)

print ''.join('| %-11s' % s for s in ['T', 'beta', 'S_rpa']), '|'
print '-'*41

for idx, T in enumerate(T_rpa_vec):

    beta = 1. / T
    g0_wk, g0_wr, g0_tr = get_g0(e_k, beta, mu, n_w, kmesh, rmesh)
    chi0_tr, chi0_wr, chi0_wk = get_chi0(g0_tr, n_w, kmesh, rmesh)    

    chi_rpa_wk = chi_wk_from_U_and_chi0_wk(chi0_wk, 0.5*U)
    
    S_rpa = chi_rpa_wk[:, k_pipi].data.sum().real # FIXME: Replace with .density() call
    S_rpa_vec[idx] = S_rpa
    
    print ''.join('| %4.4E ' % x for x in [T, beta, S_rpa]), '|'

In [ ]:
n = 1.
U = 4.

Usp, Uch = 1., 1. # initial guess

T_tpsc_vec = np.array([10., 9., 8., 7., 6., 5., 4., 3., 
                       2.5, 2.0, 1.5, 1.2, 1.0, 
                       0.8, 0.6, 0.4, 0.35, 0.3, 0.25])

S_tpsc_vec = np.zeros_like(T_tpsc_vec)
U_sp_vec = np.zeros_like(T_tpsc_vec)

print ''.join('| %-11s' % s for s in ['T', 'beta', 'Usp', 'Uch', 'docc', 'S_tpsc']), '|'
print '-'*80

for idx, T in enumerate(T_tpsc_vec):

    beta = 1. / T    
    g0_wk, g0_wr, g0_tr = get_g0(e_k, beta, mu, n_w, kmesh, rmesh)
    chi0_tr, chi0_wr, chi0_wk = get_chi0(g0_tr, n_w, kmesh, rmesh)    
    
    Usp, Uch, docc = solve_Usp_and_Uch(chi0_wk, U, n, Usp0=Usp, Uch0=Uch)
    
    chi_sp_wk = chi_wk_from_U_and_chi0_wk(chi0_wk, 0.5*Usp)
    S_tpsc = chi_sp_wk[:, k_pipi].data.sum().real

    S_tpsc_vec[idx], U_sp_vec[idx] = S_tpsc, Usp

    print ''.join('| %4.4E ' % x for x in [T, beta, Usp, Uch, docc, S_tpsc]), '|'


In [ ]:
plt.figure(figsize=(3.25*2, 5*4))
subp = [4, 1, 1]

plt.subplot(*subp); subp[-1] += 1
plt.title(r'$U = %2.2f$' % U)
plt.plot(T_rpa_vec, S_rpa_vec, 'o-', label=r'$S_{RPA}$', alpha=0.5)
plt.plot(T_tpsc_vec, S_tpsc_vec, 'o-', label=r'$S_{TPSC}$', alpha=0.5)
plt.legend(loc='best')
plt.xlabel(r'$T$')

plt.subplot(*subp); subp[-1] += 1
plt.plot(T_rpa_vec, 1./S_rpa_vec, 'o-', alpha=0.5, label=r'$S_{RPA}^{-1}$')
plt.plot(T_tpsc_vec, 1./S_tpsc_vec, 'o-', alpha=0.5, label=r'$S_{TPSC}^{-1}$')
plt.legend(loc='best')
plt.xlabel(r'$T$'); plt.grid()

plt.subplot(*subp); subp[-1] += 1
plt.plot(T_rpa_vec, 1./S_rpa_vec, 'o-', alpha=0.5, label=r'$S_{RPA}^{-1}$')
plt.plot(T_tpsc_vec, 1./S_tpsc_vec, 'o-', alpha=0.5, label=r'$S_{TPSC}^{-1}$')
plt.legend(loc='best'); plt.xlim([0, 2]); plt.ylim([-0.1, 2.5])
plt.xlabel(r'$T$'); plt.grid()

plt.subplot(*subp); subp[-1] += 1
plt.plot(T_rpa_vec, U + 0*T_rpa_vec, 'o-', alpha=0.5, label=r'$U$')
plt.plot(T_tpsc_vec, U_sp_vec, 'o-', alpha=0.5, label=r'$U_{sp}$')
plt.legend(loc='best')
plt.xlabel(r'$T$')

plt.tight_layout()
plt.savefig('figure_S_TPSC_RPA_vs_T.pdf')

# RPA phase boundary

While the TPSC fulfills the Mermin-Wagner theorem RPA does not. Instead it predicts an anti-ferro magnetic transition at finite temperature for finite interaction $U$.


In [ ]:
Tc_rpa = np.arange(0.01, 1.0, 0.05)
Uc_rpa = np.zeros_like(Tc_rpa)

for idx, T in enumerate(Tc_rpa):

    beta = 1. / T    
    g0_wk, g0_wr, g0_tr = get_g0(e_k, beta, mu, n_w, kmesh, rmesh)
    chi0_tr, chi0_wr, chi0_wk = get_chi0(g0_tr, n_w, kmesh, rmesh)    
        
    chi0_w0kpipi = chi0_wk[Idx(0), k_pipi][0, 0].real
    Uc = 2.0 / chi0_w0kpipi
    Uc_rpa[idx] = Uc
    
    print 'T, beta, Uc =', T, beta, Uc
        

In [ ]:
plt.plot(Uc_rpa, Tc_rpa, '.-')